# 1.	参照例2完成：在 www.amazon.cn 搜索python，提取搜索页面python书籍的价格和名称，至少3个页面的信息

**+ 注意是书籍而非课程；**


In [6]:
# 定义所需类

import requests
from bs4 import BeautifulSoup
import re

class amazon_python(object):
    
    def __init__(self,depth = 2):
        self.url = 'http://www.amazon.cn/s'
        self.depth = depth
        self.count = 1
        self.headers = {
            'User-Agent': 'Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1)',
            'Host': 'www.amazon.cn'
        }



    def get_data(self,url,page):
        self.params = {
            'k': 'python',
            'i': 'stripbooks',
            'page': str(page),
        }
        # try:
        response = requests.get(url=self.url,timeout = 20,params=self.params,headers = self.headers)
        response.raise_for_status()
        response.encoding = response.apparent_encoding
        
        return response.text
        # except:
        #     return''
        
        
        
        
    def parse_data(self,data,result):
        bs = BeautifulSoup(data,"html.parser")
        table = bs.find(name='span',attrs={'data-component-type':"s-search-results"}).find(name='div',attrs={"class":"s-main-slot s-result-list s-search-results sg-row"}).find_all('div',attrs = {'data-index':re.compile(r'\d+')})
        # print(table)
        for list in table:
            temp = {}
            try:
                
                temp['price']=list.find(name = 'span',attrs = {'class':"a-price"}).find(name = 'span',attrs = {'class':"a-offscreen"}).string
                temp['name']=list.find(name='span',attrs={'class':"a-size-medium a-color-base a-text-normal"}).string
                temp['rank']=self.count
                self.count += 1 
                
                result.append(temp)
            except:
                pass
            
        return result
        
        

    def printlist(self,list):
        f = '{:^4}\t{:^16}\t{:^32}'
        print(f.format('序号','价格','商品名称',chr(12888)))
        for i in range(len(list)):
            print(f.format(list[i]['rank'],list[i]['price'],list[i]['name'],chr(12888)))
            
        
        
        
        
    def main(self):
        result = []
        for i in range(self.depth):
            data = self.get_data(url=self.url,page=i+1)
            result = self.parse_data(data,result)
            
        self.printlist(result)
    

In [7]:
# 实例化并运行

if __name__ == '__main__':
    trial = amazon_python(depth = 3)
    trial.main()

 序号 	       价格       	              商品名称              
 1  	     ¥0.00      	      编程小白的第一本 Python 入门书       
 2  	     ¥24.60     	Python编程：从入门到实践（第2版）（新版上线！！！！沿袭第1版讲解清晰透彻、循序渐进的特点，并全面升级！源代码文件+视频讲解+配套编程环境，零基础自学Python一本就够！）（图灵图书）
 3  	     ¥10.99     	  Python编程快速上手——让繁琐工作自动化（异步图书）  
 4  	     ¥84.74     	Python编程实战（妙趣横生的项目编程，不同凡响的Python之旅）
 5  	     ¥55.99     	      流畅的Python (图灵程序设计丛书)      
 6  	     ¥49.90     	       Python编程轻松进阶（图灵图书）       
 7  	     ¥70.00     	       算法深潜：勇敢者的Python探险        
 8  	     ¥44.90     	 Excel+Python：飞速搞定数据分析与处理（图灵图书） 
 9  	     ¥39.99     	        Python深度学习（图灵图书）        
 10 	     ¥71.06     	   Python工匠：案例、技巧与工程实践（图灵图书）    
 11 	     ¥38.50     	Python深度学习入门：从零构建CNN和RNN【前Facebook数据科学家带你领略深度学习领域的全貌!】（图灵图书）
 12 	    ¥118.30     	        Python王者归来（增强版）         
 13 	     ¥26.99     	 Python带我起飞——入门、进阶、商业实战（博文视点图书） 
 14 	     ¥67.74     	     IT审计：用SQL+Python提升工作效率     
 15 	     ¥70.00     	   Python黑帽子：黑客与渗透测试编程之道（第2版）   
 16 	     ¥41.00     	Pyt

# 2． 参照ppt中的例子，采用scrapy 实现爬取股票数据

各个文件主要代码如下

In [1]:
#物品文件: items.py
# Define here the models for your scraped items
#
# See documentation in:
# https://docs.scrapy.org/en/latest/topics/items.html

import scrapy


class StockItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()
    code = scrapy.Field()
    name = scrapy.Field()
    latest_price = scrapy.Field()
    chg = scrapy.Field()
    change = scrapy.Field()
    volume = scrapy.Field()
    turnover = scrapy.Field()
    amplitude = scrapy.Field()
    highest = scrapy.Field()
    lowest = scrapy.Field()
    price_today = scrapy.Field()
    prec = scrapy.Field()
    volume_ratio = scrapy.Field()  
    turnover_rate = scrapy.Field()  
    pe_ratio = scrapy.Field()  
    pb_ration = scrapy.Field()  



    pass


In [ ]:
# 主爬虫文件：eastmoney.py
import scrapy
from stock.items import StockItem
import json
import urllib


class EastmoneySpider(scrapy.Spider):
    name = 'eastmoney'
    allowed_domains = ['eastmoney.com']
    start_urls = ['http://32.push2.eastmoney.com/api/qt/clist/get']


    def get_params(self,page):
        params = {
            
            'pn': str(page),
            'pz': '20',
            'po': '1',
            'np': '1',
            'ut': 'bd1d9ddb04089700cf9c27f6f7426281',
            'fltt': '2',
            'invt': '2',
            'wbp2u': '|0|0|0|web',
            'fid': 'f3',
            'fs': 'm:0 t:6,m:0 t:80,m:1 t:2,m:1 t:23,m:0 t:81 s:2048',
            'fields': 'f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f12,f13,f14,f15,f16,f17,f18,f20,f21,f23,f24,f25,f22,f11,f62,f128,f136,f115,f152',
        }
        return params


    def start_requests(self):
        requests = []
        for i in range(112):
            requests.append(scrapy.Request(url=self.start_urls[0] +'?'+ urllib.parse.urlencode(self.get_params(i+1)),headers={'Referer': 'http://quote.eastmoney.com/'}))
        return requests


    def parse(self, response):
        item = StockItem()
        data = json.loads(response.text)
        # print(data)
        for s in data['data']['diff']:
            item['code'] = s['f12']
            item['name'] = s['f14']
            item['latest_price'] = s['f15']
            item['chg'] = s['f3']
            item['change'] = s['f4']
            item['volume'] = s['f5']
            item['turnover'] = s['f6']
            item['amplitude'] = s['f7']
            item['highest'] = s['f15']
            item['lowest'] = s['f16']
            item['price_today'] = s['f17']
            item['prec'] = s['f18']
            item['volume_ratio'] = s['f10']
            item['turnover_rate'] = s['f8']
            item['pe_ratio'] = s['f9']
            item['pb_ration'] = s['f23']
            # print(item)
            yield item

        pass


In [ ]:
# 物品处理文件： pipelines.py
# Define your item pipelines here
#
# Don't forget to add your pipeline to the ITEM_PIPELINES setting
# See: https://docs.scrapy.org/en/latest/topics/item-pipeline.html


# useful for handling different item types with a single interface
from itemadapter import ItemAdapter


class StockPipeline:
    def process_item(self, item, spider):
        return item


class Stockinfopipeline:
    def open_spider(self, spider):
        self.count = 1
        self.f = open('eastmoney.txt','w',encoding='utf-8')

    def close_spider(self,spider):
        self.f.close()


    def process_item(self,item,spider):
        try:
            line =str(self.count) + '\t' + str(dict(item)) + '\n'
            self.f.write(line)
            self.count += 1
        except:
            pass
        return item

+ 在设置文件中设置了user_agent与robot协议

+ 部分结果如下

In [ ]:
1	{'code': '300894', 'name': '火星人', 'latest_price': 30.78, 'chg': 5.1, 'change': 1.49, 'volume': 24231, 'turnover': 73059315.22, 'amplitude': 6.65, 'highest': 30.78, 'lowest': 28.84, 'price_today': 29.18, 'prec': 29.19, 'volume_ratio': 2.16, 'turnover_rate': 2.15, 'pe_ratio': 44.87, 'pb_ration': 8.31}
2	{'code': '600781', 'name': '*ST辅仁', 'latest_price': 2.89, 'chg': 5.09, 'change': 0.14, 'volume': 169196, 'turnover': 48251981.0, 'amplitude': 5.82, 'highest': 2.89, 'lowest': 2.73, 'price_today': 2.74, 'prec': 2.75, 'volume_ratio': 1.72, 'turnover_rate': 4.52, 'pe_ratio': -2.2, 'pb_ration': 4.67}
3	{'code': '600048', 'name': '保利发展', 'latest_price': 18.56, 'chg': 5.06, 'change': 0.89, 'volume': 1358947, 'turnover': 2486154980.0, 'amplitude': 6.48, 'highest': 18.56, 'lowest': 17.42, 'price_today': 17.49, 'prec': 17.58, 'volume_ratio': 2.1, 'turnover_rate': 1.14, 'pe_ratio': 10.21, 'pb_ration': 1.22}
4	{'code': '600289', 'name': 'ST信通', 'latest_price': 3.12, 'chg': 5.05, 'change': 0.15, 'volume': 61870, 'turnover': 19133091.0, 'amplitude': 5.39, 'highest': 3.12, 'lowest': 2.96, 'price_today': 2.98, 'prec': 2.97, 'volume_ratio': 4.66, 'turnover_rate': 1.09, 'pe_ratio': -10.15, 'pb_ration': 1.22}
5	{'code': '600303', 'name': 'ST曙光', 'latest_price': 6.88, 'chg': 5.04, 'change': 0.33, 'volume': 145734, 'turnover': 99054723.0, 'amplitude': 4.27, 'highest': 6.88, 'lowest': 6.6, 'price_today': 6.69, 'prec': 6.55, 'volume_ratio': 0.37, 'turnover_rate': 2.16, 'pe_ratio': -22.22, 'pb_ration': 1.95}
6	{'code': '603603', 'name': '*ST博天', 'latest_price': 6.48, 'chg': 5.02, 'change': 0.31, 'volume': 124638, 'turnover': 80156393.0, 'amplitude': 4.54, 'highest': 6.48, 'lowest': 6.2, 'price_today': 6.2, 'prec': 6.17, 'volume_ratio': 0.79, 'turnover_rate': 4.67, 'pe_ratio': -6.98, 'pb_ration': -2.51}
7	{'code': '000889', 'name': 'ST中嘉', 'latest_price': 2.3, 'chg': 5.02, 'change': 0.11, 'volume': 564691, 'turnover': 127043774.5, 'amplitude': 6.85, 'highest': 2.3, 'lowest': 2.15, 'price_today': 2.19, 'prec': 2.19, 'volume_ratio': 2.1, 'turnover_rate': 6.49, 'pe_ratio': -51.29, 'pb_ration': 6.79}
8	{'code': '002200', 'name': 'ST交投', 'latest_price': 8.16, 'chg': 5.02, 'change': 0.39, 'volume': 7489, 'turnover': 6111024.0, 'amplitude': 0.0, 'highest': 8.16, 'lowest': 8.16, 'price_today': 8.16, 'prec': 7.77, 'volume_ratio': 1.47, 'turnover_rate': 0.41, 'pe_ratio': -42.15, 'pb_ration': 69.65}
9	{'code': '002022', 'name': '*ST 科华', 'latest_price': 12.38, 'chg': 5.0, 'change': 0.59, 'volume': 81933, 'turnover': 99318588.86, 'amplitude': 5.09, 'highest': 12.38, 'lowest': 11.78, 'price_today': 11.79, 'prec': 11.79, 'volume_ratio': 1.2, 'turnover_rate': 1.59, 'pe_ratio': 68.57, 'pb_ration': 1.77}
10	{'code': '603117', 'name': 'ST万林', 'latest_price': 2.73, 'chg': 5.0, 'change': 0.13, 'volume': 272672, 'turnover': 72626804.0, 'amplitude': 5.38, 'highest': 2.73, 'lowest': 2.59, 'price_today': 2.66, 'prec': 2.6, 'volume_ratio': 3.61, 'turnover_rate': 4.31, 'pe_ratio': -56.58, 'pb_ration': 0.88}
11	{'code': '600666', 'name': '*ST瑞德', 'latest_price': 1.89, 'chg': 5.0, 'change': 0.09, 'volume': 443588, 'turnover': 82014264.0, 'amplitude': 7.78, 'highest': 1.89, 'lowest': 1.75, 'price_today': 1.78, 'prec': 1.8, 'volume_ratio': 5.0, 'turnover_rate': 5.02, 'pe_ratio': -6.44, 'pb_ration': -4.72}
12	{'code': '002411', 'name': '*ST必康', 'latest_price': 10.55, 'chg': 4.98, 'change': 0.5, 'volume': 37562, 'turnover': 39627910.0, 'amplitude': 0.0, 'highest': 10.55, 'lowest': 10.55, 'price_today': 10.55, 'prec': 10.05, 'volume_ratio': 0.11, 'turnover_rate': 0.25, 'pe_ratio': 24.96, 'pb_ration': 2.03}
13	{'code': '600139', 'name': '*ST西源', 'latest_price': 4.23, 'chg': 4.96, 'change': 0.2, 'volume': 211353, 'turnover': 88025270.0, 'amplitude': 5.46, 'highest': 4.23, 'lowest': 4.01, 'price_today': 4.15, 'prec': 4.03, 'volume_ratio': 1.01, 'turnover_rate': 3.19, 'pe_ratio': -19.28, 'pb_ration': -5.0}
14	{'code': '600112', 'name': '*ST天成', 'latest_price': 4.03, 'chg': 4.95, 'change': 0.19, 'volume': 135614, 'turnover': 54296739.0, 'amplitude': 4.95, 'highest': 4.03, 'lowest': 3.84, 'price_today': 3.88, 'prec': 3.84, 'volume_ratio': 1.28, 'turnover_rate': 2.66, 'pe_ratio': 96.04, 'pb_ration': 14.26}
15	{'code': '002592', 'name': 'ST八菱', 'latest_price': 6.17, 'chg': 4.93, 'change': 0.29, 'volume': 100715, 'turnover': 60964063.28, 'amplitude': 5.1, 'highest': 6.17, 'lowest': 5.87, 'price_today': 5.87, 'prec': 5.88, 'volume_ratio': 0.82, 'turnover_rate': 3.85, 'pe_ratio': 74.46, 'pb_ration': 2.33}
16	{'code': '002211', 'name': 'ST宏达', 'latest_price': 2.98, 'chg': 4.93, 'change': 0.14, 'volume': 132990, 'turnover': 38400893.92, 'amplitude': 7.75, 'highest': 2.98, 'lowest': 2.76, 'price_today': 2.84, 'prec': 2.84, 'volume_ratio': 3.41, 'turnover_rate': 3.08, 'pe_ratio': -35.28, 'pb_ration': 37.07}
17	{'code': '300767', 'name': '震安科技', 'latest_price': 56.39, 'chg': 4.91, 'change': 2.63, 'volume': 21100, 'turnover': 116659976.16, 'amplitude': 6.31, 'highest': 56.39, 'lowest': 53.01, 'price_today': 53.05, 'prec': 53.55, 'volume_ratio': 0.99, 'turnover_rate': 1.04, 'pe_ratio': 124.76, 'pb_ration': 11.01}
18	{'code': '002881', 'name': '美格智能', 'latest_price': 42.0, 'chg': 4.9, 'change': 1.95, 'volume': 28122, 'turnover': 113650012.9, 'amplitude': 9.08, 'highest': 42.0, 'lowest': 38.39, 'price_today': 39.34, 'prec': 39.76, 'volume_ratio': 1.21, 'turnover_rate': 1.75, 'pe_ratio': 57.92, 'pb_ration': 13.17}
19	{'code': '000856', 'name': '冀东装备', 'latest_price': 9.21, 'chg': 4.9, 'change': 0.41, 'volume': 152611, 'turnover': 135626055.83, 'amplitude': 10.16, 'highest': 9.21, 'lowest': 8.36, 'price_today': 8.42, 'prec': 8.37, 'volume_ratio': 3.66, 'turnover_rate': 6.72, 'pe_ratio': 206.67, 'pb_ration': 5.26}
20	{'code': '002352', 'name': '顺丰控股', 'latest_price': 50.51, 'chg': 4.87, 'change': 2.34, 'volume': 331746, 'turnover': 1655967000.47, 'amplitude': 4.95, 'highest': 50.51, 'lowest': 48.13, 'price_today': 48.5, 'prec': 48.04, 'volume_ratio': 1.99, 'turnover_rate': 0.68, 'pe_ratio': 49.08, 'pb_ration': 3.02}
21	{'code': '002372', 'name': '伟星新材', 'latest_price': 21.11, 'chg': 7.24, 'change': 1.42, 'volume': 158072, 'turnover': 327342032.5, 'amplitude': 8.47, 'highest': 21.11, 'lowest': 19.45, 'price_today': 19.6, 'prec': 19.61, 'volume_ratio': 2.36, 'turnover_rate': 1.09, 'pe_ratio': 45.89, 'pb_ration': 7.72}
22	{'code': '603008', 'name': '喜临门', 'latest_price': 36.88, 'chg': 7.12, 'change': 2.39, 'volume': 81781, 'turnover': 291263547.0, 'amplitude': 10.4, 'highest': 36.88, 'lowest': 33.39, 'price_today': 33.5, 'prec': 33.57, 'volume_ratio': 2.35, 'turnover_rate': 2.11, 'pe_ratio': 31.72, 'pb_ration': 4.07}
23	{'code': '688331', 'name': '荣昌生物', 'latest_price': 54.65, 'chg': 7.11, 'change': 3.63, 'volume': 10901, 'turnover': 57981881.0, 'amplitude': 7.08, 'highest': 54.65, 'lowest': 51.04, 'price_today': 51.96, 'prec': 51.02, 'volume_ratio': 1.3, 'turnover_rate': 2.6, 'pe_ratio': -30.41, 'pb_ration': 5.46}
24	{'code': '002532', 'name': '天山铝业', 'latest_price': 9.14, 'chg': 7.1, 'change': 0.59, 'volume': 2621829, 'turnover': 2328726241.39, 'amplitude': 8.78, 'highest': 9.14, 'lowest': 8.41, 'price_today': 8.41, 'prec': 8.31, 'volume_ratio': 3.85, 'turnover_rate': 11.35, 'pe_ratio': 10.34, 'pb_ration': 1.85}
25	{'code': '002426', 'name': '胜利精密', 'latest_price': 2.78, 'chg': 7.09, 'change': 0.18, 'volume': 1684131, 'turnover': 452979480.5, 'amplitude': 8.66, 'highest': 2.78, 'lowest': 2.56, 'price_today': 2.56, 'prec': 2.54, 'volume_ratio': 2.35, 'turnover_rate': 5.35, 'pe_ratio': -99.34, 'pb_ration': 2.08}
26	{'code': '300986', 'name': '志特新材', 'latest_price': 39.1, 'chg': 7.04, 'change': 2.52, 'volume': 52486, 'turnover': 198841110.08, 'amplitude': 10.2, 'highest': 39.1, 'lowest': 35.45, 'price_today': 35.45, 'prec': 35.8, 'volume_ratio': 3.44, 'turnover_rate': 7.5, 'pe_ratio': 44.36, 'pb_ration': 4.9}
27	{'code': '300737', 'name': '科顺股份', 'latest_price': 11.0, 'chg': 7.03, 'change': 0.72, 'volume': 237654, 'turnover': 255179294.67, 'amplitude': 7.81, 'highest': 11.0, 'lowest': 10.2, 'price_today': 10.26, 'prec': 10.24, 'volume_ratio': 3.71, 'turnover_rate': 2.68, 'pe_ratio': 26.47, 'pb_ration': 2.31}
28	{'code': '003037', 'name': '三和管桩', 'latest_price': 12.39, 'chg': 7.02, 'change': 0.79, 'volume': 225184, 'turnover': 272433276.76, 'amplitude': 9.86, 'highest': 12.39, 'lowest': 11.28, 'price_today': 11.28, 'prec': 11.26, 'volume_ratio': 4.3, 'turnover_rate': 24.74, 'pe_ratio': 115.45, 'pb_ration': 3.61}
29	{'code': '000786', 'name': '北新建材', 'latest_price': 27.77, 'chg': 6.98, 'change': 1.8, 'volume': 289580, 'turnover': 785702242.14, 'amplitude': 8.02, 'highest': 27.77, 'lowest': 25.7, 'price_today': 25.71, 'prec': 25.8, 'volume_ratio': 3.97, 'turnover_rate': 1.77, 'pe_ratio': 14.24, 'pb_ration': 2.4}
30	{'code': '603180', 'name': '金牌厨柜', 'latest_price': 27.96, 'chg': 6.96, 'change': 1.78, 'volume': 38747, 'turnover': 104527164.0, 'amplitude': 9.5, 'highest': 27.96, 'lowest': 25.53, 'price_today': 25.74, 'prec': 25.59, 'volume_ratio': 1.65, 'turnover_rate': 2.69, 'pe_ratio': 22.8, 'pb_ration': 1.78}